# Agrupaciones y agreagaciones

Una parte esencial del análisis de datos consiste en resumirlos de forma eficiente: calcular agregaciones como ``sum()``, ``mean()``, ``median()``, ``min()``, y ``max()``, en el que un solo número da una idea de la naturaleza de un conjunto de datos del tamaño que sea. En este notebook, exploraremos las agregaciones en Pandas, desde operaciones simples similares a las que hemos visto con arrays de NumPy, hasta operaciones más sofisticadas basadas en el concepto de agrupación, con ``groupby``.

Para ayudarnos con la representación de los DataFrames, nos ayudaremos de la siguiente función (no nos centremos en cómo está implementada, simplemente utilicémosla para imprimir por pantalla DataFrames):

In [1]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Dataset de planetas

Algunas librerías tienen ciertos datasets predefinidos para realizar pruebas. En este caso, vamos a utilizar un dataset de la librería ``seaborn``, que es una librería de representación que veremos más adelante.


Este dataset tiene información sobre planetas que los astrónomos han descubierto alrededor de otras estrellas (conocidos como exoplanetas). Para descargarlo, simplemente se debería utilizar la siguiente sentencia:

In [2]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [3]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


En concreto, este dataset tiene más de 1000 exoplanetas, descubiertos hasta 2014.

## Agregaciones simples en Pandas

Comenzaremos realizando acciones sobre ``Series`` que obtengan un valor a partir de un conjunto de ellos, es decir, vamos a agregar unos cuantos valores en uno solo:

In [5]:
np.random.seed(10)
ser = pd.Series(np.random.rand(5))
ser

0    0.771321
1    0.020752
2    0.633648
3    0.748804
4    0.498507
dtype: float64

In [6]:
ser.sum()

2.6730317223936253

In [7]:
ser.mean()

0.534606344478725

Trabajando con ``DataFrame``, las agregaciones por defecto se harían por cada columna:

In [9]:
df = pd.DataFrame({'A': np.random.rand(5),
                   'B': np.random.rand(5)})
df

,A,B
0,0.224797,0.685360
1,0.198063,0.953393
2,0.760531,0.003948
3,0.169111,0.512192
4,0.088340,0.812621


In [10]:
df.mean()

A    0.288168
B    0.593503
dtype: float64

Si quisiéramos hacerlo por filas, podemos hacerlo con el parámetro ``axis``:

In [11]:
df.mean(axis='columns')

0    0.455078
1    0.575728
2    0.382239
3    0.340652
4    0.450480
dtype: float64

Los objetos ``Series`` y ``DataFrame`` incluyen todas las agregaciones comunes, como suma o media. Por otra parte, existe un método que nos hace unas cuantas agregaciones estadísticas de utilidad que nos dan información sobre el ``DataFrame``, el método ``describe()``.

Probémoslo en el dataset de planetas:

In [13]:
planets.describe()

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


Esta es una forma muy útil de comenzar a comprender las propiedades generales de un conjunto de datos. Por ejemplo, podemos ver en la columna `` año`` que, aunque los primeros exoplanetas se descubrieron ya en 1989, la mitad de todos los expolanets conocidos no se descubrieron, como poco, hasta 2010, lo cual se debe en gran parte a la misión Kepler, un telescopio espacial diseñado específicamente para encontrar planetas eclipsantes alrededor de otras estrellas.

La siguiente tabla resume algunas de las funciones de agregación más importantes de Pandas, que podemos utilizar sobre ``Series`` o ``DataFrames``:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |


Tenemos un detalle completo [aquí](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

Sin embargo, para profundizar en los datos, los agregados simples a menudo no son suficiente.

El siguiente nivel de resumen de datos es la operación ``groupby``, que nos permite calcular agregados de manera rápida y eficiente en subconjuntos de datos, tal y como vimos en SQL.

## GroupBy: Separar, Aplicar, Combinar


Las agregaciones simples pueden darnos una idea del conjunto de datos, pero normalmente necesitaremos realizar alguna agrupación condicional en función de una etiqueta o índice, lo que se implementa con la operación ``groupby``.

El nombre ``group by`` proviene del comando de SQL que vimos en su día, y su funcionamiento es similar.

### Separar, Aplicar, Combinar (split-apply-combine)

En la siguiente figura se resume este concepto, base del ``group by``:

![texto alternativo](https://drive.google.com/uc?id=10kjAL5qSFqMlkkCGZLvhCo5GcgQ5ZF0-)

Asi es como funciona el ``groupby``:

- El paso "split" (separar) implica dividir y agrupar un ``DataFrame`` según el valor de la clave especificada.
- El paso "appply" (aplicar) implica calcular alguna función, generalmente un agregado, transformación o filtrado, dentro de los grupos individuales.
- El paso "combine" (combinar) une estos resultados en una matriz de salida.

Si bien es cierto que se podría hacer manualmente usando alguna combinación de los comandos de enmascaramiento, agregación y fusión vistos en notebooks pasados, con ``group by`` podemos evitar realizar divisiones intermedias de forma explícita. Gracias al ``groupBy``, podemos realizar esto en una sola pasada sobre los datos, actualizando la suma, media, recuento, mínimo u otro agregado para cada grupo. El poder del ``groupBy`` es que abstrae estos pasos, de modo que el usuario no necesita pensar en cómo se realiza el cálculo por debajo, sino en la operación como un todo.

Para ejemplificar esto, veamos un ejemplo:

In [14]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


La operación más básica de split-apply-combine puede ser realizada con el método ``groupby()`` de los ``DataFrame``, indicando por qué columna se debe realizar esa agrupación:

In [15]:
df.groupby('key')

Es importante resaltar que lo que se devuelve no es un conjunto de ``DataFrame``, sino un objeto ``DataFrameGroupBy``.

Este objeto es donde está la magia: podríamos imaginarnos este objeto como una vista especial del ``DataFrame``, que está listo para profundizar en los grupos, pero no para realizar ningún cálculo real hasta que se aplica la agregación.
Este enfoque se denomina "lazy evaluation" (evaluación perezosa), lo que hace que los agregados comunes se puedan implementar de manera muy eficiente casi transparente para el usuario.

Para obtener un resultado, tenemos que aplicar un agregado a este objeto ``DataFrameGroupBy``, que realizará los pasos apropiados de split-combine (separación-combinación) para producir el resultado deseado.

Veamos un ejemplo con la suma, que no es más que una de las múltiples opciones que tenemos:

In [16]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


### El objeto GroupBy

El objeto ``GroupBy`` es una abstracción muy flexible.
Podríamos tratarlo como si fuera una colección de ``DataFrame``, que hace las cosas difíciles a nivel bajo, permitiéndonos abstraernos de todo ello. Veamos algunos ejemplos usando los datos de Planetas.

No obstante, antes de pasar a la apasionante parte donde analizaremos con detalle las funciones de agregado, filtrado, transformación y aplicación, vamos a presentar algunas de las otras funciones que podemos usar con ``GroupBy``:

#### Indexado de columnas

El objeto ``GroupBy`` soporta el indexado de columnas del mismo modo que un ``DataFrame``, devolviendo como resultado otro objeto ``GroupBy`` modificado:

In [17]:
planets.groupby('method')

In [18]:
planets.groupby('method')['orbital_period']

Aquí hemos seleccionado un grupo particular de ``Series`` del grupo original de ``DataFrame`` por referencia a su nombre de columna.
Al igual que con el objeto ``GroupBy``, no se realiza ningún cálculo hasta que llamamos a algún agregado en el objeto:

In [ ]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

Esto da una idea de la escala general de períodos orbitales (en días) a los que es sensible cada método.

#### Iteración sobre grupos

El objeto ``GroupBy`` soporta la iteración directa sobre grupos, devolviendo cada grupo como ``Series`` o ``DataFrame``:

In [19]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


Esto resulta ser muy útil para hacer ciertas cosas manualmente, aunque a menudo es mucho más rápido usar la funcionalidad incorporada ``apply()``, que discutiremos más adelante.

#### Métodos de propagación

A través de un poco de magia de las clases de Python, cualquier método no implementado explícitamente por el objeto ``GroupBy`` será ejecutado en función de los grupos, ya sean objetos ``DataFrame`` o ``Series``.

Por ejemplo, podríamos utilizar el método ``describe()`` de ``DataFrame`` para realizar un conjunto de agregaciones que describen cada grupo en los datos:

In [20]:
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


Esta tabla nos ayuda a comprender mejor los datos: por ejemplo, la gran mayoría de los planetas han sido descubiertos por los métodos de velocidad radial y tránsito, aunque este último solo se ha vuelto común en la última década.

Los métodos más nuevos parecen ser la variación del tiempo de tránsito y la modulación del brillo orbital, que no se utilizaron para descubrir un nuevo planeta hasta 2011.


Este es solo un ejemplo de la utilidad de los métodos de propagación.
Tenemos que tener en cuenta que se aplican a cada grupo individual, los resultados se combinan dentro de los ``GroupBy`` y se devuelven.
Al igual que antes, cualquier método válido de ``DataFrame`` o ``Series`` se puede utilizar en el objeto ``GroupBy`` correspondiente, lo que permite algunas operaciones muy flexibles.

### Agregación, filtrado, transformación y aplicación

En el paso anterior, nos hemos detenido en la agregación para la posterior combinación, pero hay más opciones disponibles.
En particular, los objetos ``GroupBy`` tienen métodos ``aggregate()``, ``filter()``, ``transform()`` y ``apply()``, que implementan eficientemente una variedad de útiles operaciones antes de combinar los datos agrupados.

Para el propósito de las siguientes subsecciones, usaremos este ``DataFrame``:

In [22]:
np.random.seed(10)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': np.random.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,9
1,B,1,4
2,C,2,0
3,A,3,1
4,B,4,9
5,C,5,0


#### Agregación

Ya conocemos las agregaciones de ``GroupBy`` con ``sum()``, ``median()`` y similares, pero tenemos una forma que nos permite más flexibilidad, el método ``aggregate()``.

Esta flexibilidad nos permite tomar una cadena, una función o una lista de las mismas y calcular todos los agregados a la vez.


Veamos un ejemplo:

In [23]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     1    5.0   9
B       1    2.5   4     4    6.5   9
C       2    3.5   5     0    0.0   0

Otro patrón útil es pasar un diccionario cuyas claves serán las columnas y los valores serán las operaciones que se aplicarán a cada una de ellas:

In [25]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,9
B,1,9
C,2,0


#### Filtrado

Una operación de filtrado permite quitar registros de un dataframe basado en las propiedades de un grupo.

Por ejemplo, podríamos querer mantener todos los grupos cuya desviación estándar sea mayor que algún valor umbral:

In [27]:
def filter_func(x):
    return x['data2'].std() > 4

# La función nos da los prints de cada paso que le pongamos:
display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      9
1   B      1      4
2   C      2      0
3   A      3      1
4   B      4      9
5   C      5      0

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  5.656854
B    2.12132  3.535534
C    2.12132  0.000000

df.groupby('key').filter(filter_func)
  key  data1  data2
0   A      0      9
3   A      3      1

La función de filtro debe devolver un valor booleano que especifique si el grupo pasa el filtrado. Aquí, debido a que el grupo A no tiene una desviación estándar mayor que 4, se elimina del resultado.

#### El método apply()

El método ``apply()`` nos permite aplicar una función arbitraria que actúa elemento a elemento sobre un ``Series`` o un ``DataFrame``.

Por ejemplo, en el caso de los ``DataFrame``, podremos utilizarla para aplicar una función creada por nosotros que le sume 5 si es mayor que 5, y si no, que se los reste:

In [37]:
def funcion_custom(x):
    if x > 5:
        return x + 5
    else:
        return x - 5
    
# Por ejemplo, lo aplicaremos sobre la masa:
planets['mass'].apply(funcion_custom)

# Si quisiéramos sobreescribir el df original:
# planets['mass'] = planets['mass'].apply(funcion_custom)

0       12.10
1       -2.79
2       -2.40
3       24.40
4       15.50
        ...  
1030      NaN
1031      NaN
1032      NaN
1033      NaN
1034      NaN
Name: mass, Length: 1035, dtype: float64

Sin embargo, no siempre es necesario realizar la función aparte. Muchas veces tratamos con funcinoes mucho más sencillas, que pueden ser escritas fácilmente en una línea de código. En estos casos, existe una forma más ágil de crear funciones, gracias a la sentencia ``lambda``. Con ``lambda`` podemos crear funciones sencillas y de una única expresión.

Las funciones ``lambda`` **son anónimas**, no tienen un nombre que las identifique, simplemente se ejecutan el código de la función que declaremos. 

La sintaxis de una función `lambda` es:
```Python
lambda argumentos: expresion
```

Vale. ¿Y por qué me cuentas esto ahora?

Porque son muy útiles como argumento de métodos que reciban funciones, como es este caso o el caso de las transformaciones que veremos justo después.

Veamos cómo podríamos realizar lo de antes con una función lambda y entenderemos mejor cómo funciona:

In [38]:
planets['mass'].apply(lambda x: x + 5 if x>5 else x-5)

0       12.10
1       -2.79
2       -2.40
3       24.40
4       15.50
        ...  
1030      NaN
1031      NaN
1032      NaN
1033      NaN
1034      NaN
Name: mass, Length: 1035, dtype: float64

Puede parecer un poco lioso al principio, pero en cuanto nos familiaricemos con ella, la usaremos mucho. De hecho, este método apply, combinado con la función lambda, se usará mucho en el tratamiento de variables.

Otros ejemplos podrían ser conversión de unidades, tratamiento de texto, quitar símbolos de monedas para quedarnos con el valor numérico... A continuación, veamos un ejemplo donde convertimos a string un numérico y le añadimos un texto, en este caso añadiremos las unidades Me a la masa, que representarán las unidades en relación a la masa de la Tierra:

In [43]:
planets['mass'].apply(lambda x: str(x) + " Me")

0        7.1 Me
1       2.21 Me
2        2.6 Me
3       19.4 Me
4       10.5 Me
         ...   
1030     nan Me
1031     nan Me
1032     nan Me
1033     nan Me
1034     nan Me
Name: mass, Length: 1035, dtype: object

También podríamos utilizarlo con ``DataFrames``, done podremos especificar si queremos recorrerlofila a fila (axis=1) o columna a columna (por defecto, axis=0).

Veamos un ejemplo donde creamos una nueva columna donde hacemos la multiplicación de 'masa' y 'distancia', lo convertimos a string y le añadimos una marca final:

In [60]:
planets.apply(lambda x: str(x['mass'] * x['distance']) + " z", axis = 1)

0                   549.54 z
1       125.85950000000001 z
2                   51.584 z
3                 2146.028 z
4                 1254.435 z
                ...         
1030                   nan z
1031                   nan z
1032                   nan z
1033                   nan z
1034                   nan z
Length: 1035, dtype: object

Una vez presentados a nuestros amigos ``apply()`` y ``lambda``, podemos combinarlos con lo que estamos viendo en este notebook, las agrupaciones con objetos Pandas.

La función debería tomar un ``DataFrame`` y devolver un objeto de Pandas o un escalar, la operación de combinación se adaptara al tipo de salida devuelta.

Por ejemplo, podríamos utilizar un ``apply()`` para normalizar del siguiente modo (aunque como más lo utilizaremos será con ``Series``, como las columnas):

In [75]:
df.groupby('key').apply(lambda x: x['data1']/df['data2'].sum())

key   
A    0    0.000000
     3    0.130435
B    1    0.043478
     4    0.173913
C    2    0.086957
     5    0.217391
Name: data1, dtype: float64

Del mismo modo, podríamos obtenerlo creándonos una función aparte:

In [69]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,9
1,B,1,4
2,C,2,0
3,A,3,1
4,B,4,9
5,C,5,0
,key,data1,data2
0,A,0.000000,9
1,B,0.076923,4
2,C,inf,0


El método ``apply()`` sobre un ``GroupBy`` es bastante flexible, el único criterio es que la función reciba un DataFrame y devuelva un objeos pandas o escalar, lo que hagas por el medio depende de ti.

#### Transformación

Si bien la agregación debe devolver una versión reducida de los datos, la transformación puede devolver alguna versión transformada de los datos completos para recombinarlos.

Para tal transformación, la salida tiene la misma forma que la entrada.
Un ejemplo común es centrar los datos restando la media del grupo:

In [ ]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


### Especificando la clave por la que separar los datos

En los simples ejemplos presentados anteriormente, dividimos el ``DataFrame`` en base a un solo nombre de columna.

Esta es solo una de las muchas opciones mediante las cuales se pueden definir los grupos. A continuación, veremos algunas otras opciones para especificar cómo agrupar.

#### Claves de agrupción por lista, array, Series o Index


La clave puede ser cualquier serie o lista cuya longitud coincida con la del ``DataFrame``.

Por ejemplo:

In [76]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

df
  key  data1  data2
0   A      0      9
1   B      1      4
2   C      2      0
3   A      3      1
4   B      4      9
5   C      5      0

df.groupby(L).sum()
   data1  data2
0      7      9
1      4      5
2      4      9

Por supuesto, esto significa que hay otra forma más detallada de lograr el ``df.groupby('key')`` de antes:

In [78]:
df.groupby(df['key']).sum()

,data1,data2
key,,
A,3,10
B,5,13
C,7,0


#### Mapeando grupos mediante diccionarios y Series

Otro método sería proporcionar un diccionario que asigne los índices a las claves de grupo:

In [79]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data1,data2
key,,
A,0,9
B,1,4
C,2,0
A,3,1
B,4,9
C,5,0
,data1,data2
consonant,12,13
vowel,3,10


#### Cualquier función de Python

Parecido al mapeo, puedes pasarle cualquier función de Python que tome el índice como entrada y de como salida el grupo:

In [80]:
display('df2', 'df2.groupby(str.lower).mean()')

,data1,data2
key,,
A,0,9
B,1,4
C,2,0
A,3,1
B,4,9
C,5,0
,data1,data2
a,1.5,5.0
b,2.5,6.5


#### Un alista de varios campos

Además, podremos agrupar en función de uno o más campos, obteniendo como salida un ``DataFrame`` con multi-index:

In [81]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,vowel,1.5,5.0
b,consonant,2.5,6.5
c,consonant,3.5,0.0


Como extra, si tenemos un ``DataFrame`` con dos índices, podríamos pasar uno de ellos a columnas con el método ``.unstack()``. En este caso, al tener ya columnas, nos generará un multi-indice en las columnas:

In [86]:
df2.groupby([str.lower, mapping]).mean().unstack()

data1           data2      
  consonant vowel consonant vowel
a       NaN   1.5       NaN   5.0
b       2.5   NaN       6.5   NaN
c       3.5   NaN       0.0   NaN

### Ejemplo de agrupación

Como ejemplo de esto, en unas líneas de código de Pyhton podríamos poner todo junto y contar los planetas descubiertos por método y década:

In [84]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


Esto muestra el poder de combinar muchas de las operaciones que hemos discutido hasta este punto cuando miramos conjuntos de datos reales.
Gracias a ello podemos comprender, de forma aproximada, cuándo y cómo se han descubierto los planetas en las últimas décadas.

Aquí sugeriría que profundizásemos en estas pocas líneas de código y evaluásemos los pasos individuales para asegurarnos de que comprendemos exactamente lo que estamos haciendo con el resultado.

Sin duda, es un ejemplo algo complicado, pero comprender estas piezas nos ayudará a realizar en el futuro acciones de manera similar sobre nuestros propios datos.

### Ordenación

Tan importante como agrupar puede ser ordenar, una acción que suele ir de la mano de la aagrupación en muchos casos.

Para ordenar utilizaremos el método ``sort_values()`` para ordenar valores y ``sort_index()`` para ordenar índices.

Cuando lo usamos con ``DataFrames``, ambos métodos recibirán como primer parámetro la columna o lista de columnas (en orden de prioridad) por las que realizar la ordenación, mientras que en el caso de los ``Series``, no será necesario pasarles ninguno debido a que sólo hay una csoa que ordenar.

Ambos aceptarán el parámetro ``ascending``, que para los ``DataFrames`` será una lista de booleanos en función de si queremos que cada una de las columnas por las que ordenamos sea ordenada de forma ascendente o descendente. Esta lista deberá ser del mismo tamaño que las columnas utilizadas para agrupar. En cuanto a los ``Series``, este parámetro solo recibirá un valor booleano.

In [95]:
np.random.seed(10)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': np.random.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df.sort_values(by=['data1', 'data2'], ascending=[False, True])

,key,data1,data2
5,C,5,0
4,B,4,9
3,A,3,1
2,C,2,0
1,B,1,4
0,A,0,9


In [96]:
df['data1'].sort_values(ascending=False)

5    5
4    4
3    3
2    2
1    1
0    0
Name: data1, dtype: int64